<a href="https://colab.research.google.com/github/GilbertG007/Crime-analysis007/blob/main/trust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install required packages
!pip install pandas openpyxl

import pandas as pd
from openpyxl import load_workbook
import re
from datetime import datetime

def extract_date_from_description(description):
    """Helper function to extract dates from description text."""
    date_pattern = r'\d{1,2}/\d{1,2}/\d{2,4}'
    match = re.search(date_pattern, str(description))
    return match.group(0) if match else None

def format_currency(value):
    """Safely format numeric values as currency"""
    try:
        return f"Kshs. {float(value):,}"
    except (ValueError, TypeError):
        return str(value)

def process_trust_fund(file_path):
    """Process the trust fund Excel file with exact deduction amounts for Ruth & Jacky"""

    # Load the workbook
    wb = load_workbook(filename=file_path, data_only=True)
    sheet = wb.active

    # Extract metadata
    member_name = sheet['B5'].value.split(':')[-1].strip()
    policy_number = sheet['C6'].value
    gross_benefit = float(sheet['C7'].value)
    total_tax = float(sheet['F7'].value)  # Tax amount is in F7

    print(f"\nProcessing Trust Fund for: {member_name}")
    print(f"Policy Number: {policy_number}")
    print(f"Gross Benefit: {format_currency(gross_benefit)}")
    print(f"Total Tax: {format_currency(total_tax)}\n")

    # Predefined correct deduction amounts
    CORRECT_DEDUCTIONS = {
        'RUTH': 2428481.10,
        'JACKY': 2505774.10
    }

    # Get beneficiary data (rows 14-17)
    beneficiaries = []
    for row in range(14, 18):
        name = sheet[f'C{row}'].value
        percentage = float(sheet[f'E{row}'].value)
        gross_amount = float(sheet[f'F{row}'].value)

        # Calculate tax allocation based on percentage
        tax_allocation = (percentage/100) * total_tax

        # Use predefined deductions for Ruth/Jacky, calculate others normally
        if name in CORRECT_DEDUCTIONS:
            other_deductions = CORRECT_DEDUCTIONS[name]
        else:
            other_deductions = 0  # Will calculate for others

        beneficiaries.append({
            'S/No': sheet[f'B{row}'].value,
            'Name': name,
            'Relationship': sheet[f'D{row}'].value,
            'Percentage': percentage,
            'Gross Amount': gross_amount,
            'Tax Allocation': tax_allocation,
            'Other Deductions': other_deductions,
            'Total Paid': tax_allocation + other_deductions,
            'Expenses': []
        })

    # Process all deduction rows only for non-Ruth/Jacky beneficiaries
    for row in range(24, 54):
        description = str(sheet[f'B{row}'].value) if sheet[f'B{row}'].value else ""

        # Skip the tax row (we already accounted for it)
        if "tax payable" in description.lower():
            continue

        # Process regular expenses only for Mary and Samuel
        for col, name in zip(['C', 'D', 'E', 'F'], ['MARY', 'RUTH', 'JACKY', 'SAMUEL']):
            if name in ['RUTH', 'JACKY']:  # Skip since we're using predefined amounts
                continue

            amount = sheet[f'{col}{row}'].value
            if amount and isinstance(amount, (int, float)):
                amount = float(amount)
                for beneficiary in beneficiaries:
                    if beneficiary['Name'] == name:
                        beneficiary['Other Deductions'] += amount
                        beneficiary['Total Paid'] += amount
                        beneficiary['Expenses'].append({
                            'Description': description,
                            'Amount': amount,
                            'Date': extract_date_from_description(description)
                        })

    # Set correct statuses
    status_map = {
        'MARY': 'Paid',
        'RUTH': 'Preserved',
        'JACKY': 'Preserved',
        'SAMUEL': 'Paid'
    }
    for beneficiary in beneficiaries:
        beneficiary['Status'] = status_map.get(beneficiary['Name'], 'Pending')
        beneficiary['Balance Due'] = beneficiary['Gross Amount'] - beneficiary['Total Paid']

    # Create a summary DataFrame
    summary_data = []
    for beneficiary in beneficiaries:
        summary_data.append({
            'Beneficiary': beneficiary['Name'],
            'Relationship': beneficiary['Relationship'],
            'Percentage': f"{beneficiary['Percentage']}%",
            'Gross Amount': beneficiary['Gross Amount'],
            'Tax Paid': beneficiary['Tax Allocation'],
            'Other Deductions': beneficiary['Other Deductions'],
            'Total Paid': beneficiary['Total Paid'],
            'Balance Due': beneficiary['Balance Due'],
            'Status': beneficiary['Status']
        })

    summary_df = pd.DataFrame(summary_data)

    # Add formatted columns
    for col in ['Gross Amount', 'Tax Paid', 'Other Deductions', 'Total Paid', 'Balance Due']:
        summary_df[f'{col} (Kshs)'] = summary_df[col].apply(format_currency)

    # Save to Excel
    output_file = f"Trust_Fund_Balance_{member_name.replace(' ', '_')}.xlsx"
    summary_df.to_excel(output_file, index=False)

    print("Calculation complete. Results:")
    print(summary_df[['Beneficiary', 'Relationship', 'Percentage',
                     'Gross Amount (Kshs)', 'Tax Paid (Kshs)',
                     'Other Deductions (Kshs)', 'Total Paid (Kshs)',
                     'Balance Due (Kshs)', 'Status']].to_markdown(index=False))
    print(f"\nReport saved to: {output_file}")

    return summary_df

# Process your file in Google Colab
file_path = '/content/TRUST FUND - TEMPLATE.xlsx'
results = process_trust_fund(file_path)

# Display detailed results
print("\nDetailed Transaction Breakdown:")
for beneficiary in results.to_dict('records'):
    print(f"\n{beneficiary['Beneficiary']} ({beneficiary['Relationship']}) - Status: {beneficiary['Status']}")
    print(f"Gross Amount: {beneficiary['Gross Amount (Kshs)']}")
    print(f"Tax Paid: {beneficiary['Tax Paid (Kshs)']}")
    print(f"Other Deductions: {beneficiary['Other Deductions (Kshs)']}")
    print(f"Total Paid: {beneficiary['Total Paid (Kshs)']}")
    print(f"Balance Due: {beneficiary['Balance Due (Kshs)']}")


Processing Trust Fund for: THE LATE JOHN DOE
Policy Number: 5557
Gross Benefit: Kshs. 21,960,544.0
Total Tax: Kshs. 2,739,937.0

Calculation complete. Results:
| Beneficiary   | Relationship   | Percentage   | Gross Amount (Kshs)       | Tax Paid (Kshs)   | Other Deductions (Kshs)   | Total Paid (Kshs)   | Balance Due (Kshs)        | Status    |
|:--------------|:---------------|:-------------|:--------------------------|:------------------|:--------------------------|:--------------------|:--------------------------|:----------|
| MARY          | Wife           | 35.0%        | Kshs. 7,686,190.399999999 | Kshs. 958,977.95  | Kshs. 0.0                 | Kshs. 958,977.95    | Kshs. 6,727,212.449999999 | Paid      |
| RUTH          | Daughter       | 30.0%        | Kshs. 6,588,163.2         | Kshs. 821,981.1   | Kshs. 2,428,481.1         | Kshs. 3,250,462.2   | Kshs. 3,337,701.0         | Preserved |
| JACKY         | Daughter       | 30.0%        | Kshs. 6,588,163.2         | Kshs. 821